In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 500 observations

In [2]:
n_sim, nobs, expo = 1000, 500, 250
err_pnl, popsize = 3e5, 8000

In [3]:
# Setting the true model
f1, f2 = bs.loss_model("Weibull", ["k1", "β1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])

f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

%time PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

CPU times: user 39.6 s, sys: 27.1 ms, total: 39.7 s
Wall time: 39.7 s


array([-169.54582835, -125.42571575,  -54.56573389])

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [4]:
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
def fit_one(k, f, f_name,  prior, X):
    trace, log_marg, DIC, WAIC = bs.smc(X, f, popsize, prior, verbose = False)
    PnLs = np.array(f.PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
    cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
    return([k, f_name , nobs, log_marg, trace["γ"].mean(), parms_true[-1] ] + cap.tolist() + true_cap.tolist())
def fit_all(k):
    print("simulation #"+ str(k))
    col_names = ["sim", "model_name", "nobs", "log_marg", "γ_map", "γ_true", "cap005", "cap01", "cap05", "cap005_true", "cap01_true", "cap05_true" ]
    df_temp = pd.DataFrame([fit_one(k, fs[i], f_names[i], prior_spliced_model[i], Xs[k]) for i in range(len(fs))], columns = col_names)
    df_temp['posterior_probability'] = np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)) / \
    np.sum(np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)))  
    γ_BMA = np.dot(df_temp['posterior_probability'].values, df_temp['γ_map'])
    cap_bma = np.dot(df_temp['posterior_probability'].values, df_temp[["cap005", "cap01", "cap05"]].values)
    BMA_df = pd.DataFrame([[k, "BMA" , nobs, None, γ_BMA, parms_true[-1] ] + cap_bma.tolist() + true_cap.tolist()],
            columns = col_names)
    df_temp = pd.concat([df_temp, BMA_df])
    return(df_temp)
%time res = Parallel(n_jobs= 50)(delayed(fit_all)(k) for k in range(n_sim))
pd.concat(res).to_csv("../../Data/Simulations/simu_wellspec_"+str(nobs)+".csv", sep=',')

/home/pgoffard/miniconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 2min 37s, sys: 5.72 s, total: 2min 43s
Wall time: 4h 11min 43s


In [5]:
pd.concat(res)

,sim,model_name,nobs,log_marg,γ_map,γ_true,cap005,cap01,cap05,cap005_true,cap01_true,cap05_true,posterior_probability
0,0,Weibull_Log-Logistic,500,-395.804474,1.908442,1.5,-411.128935,-270.364388,-84.815071,-169.545828,-125.425716,-54.565734,2.837346e-01
1,0,Weibull_Lomax,500,-395.250382,1.609427,1.5,-218.559162,-151.210244,-53.197721,-169.545828,-125.425716,-54.565734,4.938000e-01
2,0,Weibull_Inverse-Weibull,500,-396.047741,2.278764,1.5,-513.356465,-332.860779,-110.962150,-169.545828,-125.425716,-54.565734,2.224654e-01
3,0,Inverse-Weibull_Log-Logistic,500,-416.814063,0.601580,1.5,-459.211184,-289.244449,-77.369198,-169.545828,-125.425716,-54.565734,2.130902e-10
4,0,Inverse-Weibull_Lomax,500,-415.536253,0.204253,1.5,-262.403483,-175.547350,-55.451665,-169.545828,-125.425716,-54.565734,7.647320e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,999,Weibull_Inverse-Weibull,500,-370.665818,2.851018,1.5,-233.130217,-152.723323,-41.408368,-169.545828,-125.425716,-54.565734,3.203569e-01
3,999,Inverse-Weibull_Log-Logistic,500,-385.573883,2.772482,1.5,-182.625994,-113.807948,-19.173438,-169.545828,-125.425716,-54.565734,1.074345e-07
4,999,Inverse-Weibull_Lomax,500,-385.621967,2.593172,1.5,-153.238279,-97.920461,-18.034123,-169.545828,-125.425716,-54.565734,1.023908e-07
5,999,Inverse-Weibull_Inverse-Weibull,500,-385.682896,2.900940,1.5,-217.767389,-134.620021,-29.201068,-169.545828,-125.425716,-54.565734,9.633858e-08
